In [1]:
import torch
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ehess\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd
# import spacy
# import torch

df = pd.read_csv("contrib_from_csv.csv", sep = ",", dtype= str)

# echantillon de 10000 pour tester
df = df.sample(10000)

In [3]:
df

,Catégorie,Date de réception,Code postal,Code INSEE,Numéro d'ordre arbitraire,Type Graphie TT,Numéro de page,Numéro séquentiel,Contribution,joined_id
158262,CC,190225,79230,79216,11399,MD,36,12,Je recrute\nPoints de satisfaction\nAdministra...,CC_79230_190225_79216_MD_11399
61667,CC,190315,35170,35047,18231,D,41,25,Contact: Jane DOE Message : - Sortir de l'UE. ...,CC_35170_190315_35047_MD_18231
50281,CC,190225,53000,53130,10028,D,143,83,CONTRIBUTIONS AU GRAND DÉBAT NATIONAL QUESTION...,CC_53000_190225_53130_MD_10028
142631,CC,190227,91280,91573,13228,MD,5,2,Fin des pages écrites,CC_91280_190227_91573_MD_13228
192341,CC,190225,29200,29019,06646,MD,241,329,retraiteretraite revenir à la défiscalisation ...,CC_29200_190225_29019_MD_06646
...,...,...,...,...,...,...,...,...,...,...
213594,CC,190225,29660,29023,06755,MD,12,7,problème de retraite siège le seuil des retrai...,CC_29660_190225_29023_MD_06755
42584,CC,190301,91200,91027,14372,MD,115,37,Nous nous proposons de recueillir vos idées et...,CC_91200_190301_91027_MD_14372
177477,CC,190225,78100,78551,02737,MD,39,33,protection de service case de ce système qui a...,CC_78100_190225_78551_MD_02737
176576,CC,190225,78840,78255,02679,MD,3,1,Un très petit résumé de la situation de la Fra...,CC_78840_190225_78255_MD_02679


In [4]:
import spacy

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.device = "cuda"

In [7]:
spacy.prefer_gpu()

C:\Users\ehess\anaconda3\envs\py310\lib\site-packages\torch\cuda\__init__.py:350: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  torch._C._cuda_setDevice(device)


True

In [8]:
import GPUtil


In [9]:
is_using_gpu = spacy.prefer_gpu()

if is_using_gpu:
    print("Using GPU!")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
    print("GPU Usage")
    GPUtil.showUtilization()

Using GPU!
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  7% | 29% |


In [5]:
xob_process_breakline = lambda x: x.replace("\x0b", " ")
n_processbreaklines = lambda x: x.replace("\n", " ")

In [6]:
df["clean_xob"] = df["Contribution"].apply(xob_process_breakline)
df["clean_n"] = df["clean_xob"].apply(n_processbreaklines)

In [12]:
# pd.set_option("display.max_colwidth", None)

In [7]:
# TODO remove "illisible"

documents = df.clean_n.tolist()
documents[0]

'Je recrute Points de satisfaction Administration réferente Difficultés rencontrées Administration réferente'

In [8]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

final_stopwords_list = list(fr_stop) + list(en_stop) + ["\n"]

def remove_stopwords(text):
    stopwords = final_stopwords_list
    text = [word for word in text if word not in stopwords]
    return text

In [9]:
nostopword = remove_stopwords(documents)

In [91]:
# print(df['cleaned'])

142722    31.12.2018 . indexation retraites niveau infla...
27066     révolte gilets jaunes mérite réveiller conscie...
128861    10 janvier 2019 niggemann patrick 116 rue past...
30479     objet : propositions sujets cadre grand débat ...
125689    grand débat national " debat éclairer décision...
                                ...                        
109900                                                  NaN
222554                                                  NaN
93436                                                   NaN
214835                                                  NaN
143696                                                  NaN
Name: cleaned, Length: 10000, dtype: object


In [10]:
from top2vec import Top2Vec

ImportError: numpy.core.multiarray failed to import

In [85]:
model = Top2Vec(lemmatized)

2023-04-11 16:25:29,343 - top2vec - INFO - Pre-processing documents for training
2023-04-11 16:25:33,252 - top2vec - INFO - Creating joint document/word embedding
2023-04-11 16:26:45,189 - top2vec - INFO - Creating lower dimension embedding of documents
2023-04-11 16:27:14,975 - top2vec - INFO - Finding dense areas of documents
2023-04-11 16:27:15,506 - top2vec - INFO - Finding topics


In [86]:
topic_words, word_scores, topic_nums = model.get_topics()

In [87]:
topic_words

array([['gilet', 'longtemps', 'peuple', ..., 'beaucoup', 'dormir',
        'decennie'],
       ['pollution', 'electrique', 'production', ..., 'agricole',
        'installation', 'fabrication'],
       ['senateur', 'privilege', 'ministre', ..., 'seance', 'voyage',
        'salair'],
       ...,
       ['conditionnel', 'appel', 'victime', ..., 'naturel', 'justice',
        'tenue'],
       ['page', 'resumer', 'mail', ..., 'lundi', 'message', 'ville'],
       ['conditionnel', 'liberation', 'penal', ..., 'agresseur', 'tenir',
        'estime']], dtype='<U15')

In [5]:
rows, columns = df.shape
print(rows//1000)


225


In [22]:
newline = "\n"
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

nlp = spacy.load("fr_core_news_sm")

final_stopwords_list = list(fr_stop) + list(en_stop)


tokenize = lambda x : nlp(x)
# out = random_sample.apply(tokenize)
allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]

def preprocess():
    nlp = spacy.load("fr_core_news_sm", disable = ["parser", "ner"])
    nlp.max_length = 3000000 #this is possible as we're not using parser or ner.
    tokens = nlp(text) # Converts the text to a list of tokens (strings).
#     tokens = [token.lower_ for token in tokens] # Lowercases all tokens.
    lemmatized_lines = []
    for token in tokens :
        sentences = (sent.text)
        lemmatized_lines.append(token.lemma_)

#         lemmatized_lines.append(token.lemma if token.pos_ in allowed_postag)

    out = " ".join(lemmatized_lines)
#     out = remove_stopwords(out)
#     stopwords = final_stopwords_list
#     text = [word for word in tokens if word not in stopwords] #remove the stopwords
#     print(text)
    return out


# text = FILE.read()    
# doc = nlp(text)
# with open("FILEE.xml",'w') as fp:  
#      for sent in doc.sents:
#          sentences = (sent.text)
#          for match_id, start, end in phrase_matcher(nlp(sentences)):  
#              if nlp.vocab.strings[match_id] in ["subordoT"]:
#                 fp.write("%s\n" % sentences)

In [17]:
df["clean_n"][18927]

'Collectif des Petits Patrons et Artisans - Antenne Falaise Relais local du Collectif des Petits Patrons et Artisans, nous remercions la municipalité en générai et le Maire en particulier de prendre le temps de relayer les revendications des citoyens. Vous êtes une courroie de transmission essentielle entre les Administrés et l\'exécutif, parfois devenu sourd à notre bon sens "paysan". Les Petits Patrons et Artisans sont des gens simples, travailleurs, entrepreneurs en bâtiment, chauffagistes, plombiers, peintres, électriciens, vitriers, artisans, agriculteurs, commerçants, distributeurs, transporteurs, ambulanciers, consultants, restaurateurs, petits industriels, sociétés de services informatiques, développeurs web, entreprises digitales, attachés de presse, sociétés de communication, entreprises de nettoyage, de logistique, de stockage, de déménagement, paysagistes, sociétés de jardinage... Les petites entreprises sont donc présentes partout en France, notamment à Falaise et dans ses

In [23]:
df["cleaned"] = df["clean_n"][0:2].apply(preprocess)

NameError: name 'token' is not defined

In [ ]:
df

In [36]:
# lower = lambda x : [token.lower_ for token in x] # Lowercases all tokens.

In [37]:
# df["cleaned"] = df["cleaned"].apply(lower)

AttributeError: 'str' object has no attribute 'lower_'

In [90]:
def lemmatize(docs, allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]):
  '''
  Performs lemmization of input documents.
  Args:
    - docs: list of strings with input documents
    - allowed_postags: list of accepted Part of Speech (POS) types
  Output:
    - list of strings with lemmatized input
  '''
  nlp = spacy.load("fr_core_news_sm", disable = ["parser", "ner"])
  nlp.max_length = 3000000 #this is possible as we're not using parser or ner.
  lemmatized_docs = []
  lemmatized_lines = []

#   listdocs = docs.tolist()
#   for doc in listdocs:
  listoflines = [nlp(line) for line in docs]
#     docs = nlp(docs)
  tokens = []
  for line in listoflines :
    listtokens = [token.lemma for token in line if token.pos_ in allowed_postag]
#       for token in doc:
#         if token.pos_ in allowed_postags:
#         tokens.append(token.lemma_)
  lemmatized_lines.append(" ".join(listtokens))

  return (lemmatized_docs)